In [ ]:
from netCDF4 import Dataset
from matplotlib import pyplot as plt
import numpy as np
import datetime
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap
import os

In [ ]:
start_time_era5 = datetime.datetime(1900, 1, 1, 0, 0, 0)
lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 50, 110, 70, 83

In [ ]:
lat1, lat2 = (90-lat_max_era5)*4, (90-lat_min_era5)*4+1
lon1, lon2 = lon_min_era5*4, lon_max_era5*4+1

In [ ]:
file = '/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_2020-09.nc'
data = Dataset(file, 'r')

longitude = np.array(data.variables['longitude'][lon_min_era5*4:lon_max_era5*4+1])
latitude = np.array(data.variables['latitude'][(90-lat_max_era5)*4:(90-lat_min_era5)*4+1])
lon_grid, lat_grid = np.meshgrid(longitude, latitude)

In [ ]:
def drawing(u, v, t=None, lon = lon_grid, lat = lat_grid, save=False):
    fig = plt.figure(figsize=(12, 12), dpi=300)
    
    m = Basemap(width=1200000, height=1500000,
                resolution='l', projection='aea',
                lat_1=60, lat_2=65, lon_0=80, lat_0=77)
    m.drawcoastlines()
    m.fillcontinents(color='grey', lake_color='white')
    m.drawparallels(np.arange(-80., 90., 10.), labels=[False, True, True, False])
    m.drawmeridians(np.arange(-180., 180., 20.), labels=[True, True, False, True])
    m.drawmapboundary(fill_color='white')
    
    # Преобразование координат для карты
    x, y = m(lon, lat)
    
    # Вычисление скорости ветра
    speed = np.sqrt(u**2 + v**2)
    
    # Отображение скорости ветра
    speed_plot = m.pcolormesh(x, y, speed, shading='gouraud', cmap='jet', vmin=0)
    cbar=plt.colorbar(speed_plot, label='Wind speed (m/s)')
    # cbar.set_ticks([0,2,4,6,8,10,12])
    
    # Отображение направления ветра
    m.quiver(x[::4, ::4], y[::4, ::4], u[::4, ::4], v[::4, ::4], scale=30, scale_units='inches')
    # plt.title(f'Wind speed {t.date()}')
    
    plt.show()
    plt.close('all')

In [ ]:
year = 2022
month = 9

In [ ]:
file = f'/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
u10 = np.asarray(data.variables['u10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])
u10.shape

In [ ]:
v10 = np.asarray(data.variables['v10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])
v10.shape

In [ ]:
u10.shape, v10.shape

In [ ]:
list_u10 = np.split(u10, u10.shape[0]/24, axis=0)
array_u10 = np.array([np.mean(a, axis=0) for a in list_u10])

list_v10 = np.split(v10, v10.shape[0]/24, axis=0)
array_v10 = np.array([np.mean(a, axis=0) for a in list_v10])

In [ ]:
# lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 50, 110, 70, 83
border_north = 78
border_south = 73
border_east = 85
border_west = 70

In [ ]:
longitude[(border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]

In [ ]:
latitude[(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1]

In [ ]:
u10_upd = u10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
              (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
v10_upd = v10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
              (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
u10_upd.shape, v10_upd.shape

In [ ]:
list_u10_upd = np.split(u10_upd, u10_upd.shape[0]/24, axis=0)
array_u10_upd = np.array([np.mean(a, axis=0) for a in list_u10_upd])

list_v10_upd = np.split(v10_upd, v10_upd.shape[0]/24, axis=0)
array_v10_upd = np.array([np.mean(a, axis=0) for a in list_v10_upd])

In [ ]:
p5 = 0
p5u = 0
p5v = 0
p7 = 0
for i in range(len(list_u10_upd)):
    print(i+1, np.mean(array_u10_upd[i]), np.mean(array_v10_upd[i]))
    speed = np.sqrt(np.mean(array_u10_upd[i])**2 + np.mean(array_v10_upd[i])**2)
    if (speed > 5) and (np.mean(array_u10_upd[i]) < 0) and (np.mean(array_v10_upd[i]) < 0):
        p5 +=1
    
    if (speed > 7) and (np.mean(array_u10_upd[i]) < 0) and (np.mean(array_v10_upd[i]) < 0):
        p7 += 1
    
    if np.mean(array_u10_upd[i]) < -5:
        p5u += 1
    
    if np.mean(array_v10_upd[i]) < -5:
        p5v += 1

print(p5, p7, p5u, p5v)

In [ ]:
array_u10_upd.shape

In [ ]:
start_date = datetime.datetime(2015, 7, 1)
finish_date = datetime.datetime(2015, 8, 3)
dates = [(start_date + datetime.timedelta(days=i)).date() for i in range((finish_date - start_date).days)]

In [ ]:
(start_date + datetime.timedelta(days=31)).date()

In [ ]:
[dates[i*3+1] for i in range(len(dates)//3)]

In [ ]:
print([np.mean(array_u10_upd[i]) for i in range(len(list_u10_upd))])

In [ ]:
north_2015_75E = [75.75, 75.625, 75.5, 76.0, 76.375, 76.5, 76.875, 76.875, 77.125, 77.625, 77.875]

In [ ]:
plt.plot(dates[:-4], [-np.mean(array_u10_upd[i]) for i in range(len(list_u10_upd)-2)]);
plt.axhline(y=5, color='red', linestyle='--', linewidth=1.5);
plt.xticks(rotation=45);

In [ ]:
plt.plot([dates[i*3+1] for i in range((len(dates)-2)//3)], [north_2015_75E[i] - north_2015_75E[i-1] for i in range(1, len(north_2015_75E))]);
plt.xticks(rotation=45);

In [ ]:
plt.plot([dates[i*3+1] for i in range((len(dates)-1)//3)], [north_2015_75E[i] for i in range(len(north_2015_75E)-1)]);
plt.xticks(rotation=45);

In [ ]:
u10_mean = np.mean(u10, axis=0)
v10_mean = np.mean(v10, axis=0)
u10_mean.shape, v10_mean.shape

In [ ]:
drawing(u10_mean, v10_mean)